## Imports

In [ ]:
# Calculations
import math
import numpy as np

# Requests & Files
import re
import requests
import json
import untangle
from requests_toolbelt.multipart import decoder

# Imagery
import cv2
import mercantile
from moviepy.editor import *

# Graphic widget
from IPython.display import HTML, clear_output
from halo import HaloNotebook as Halo
import time # for delivery waiting function

## Utility functions

In [ ]:
# Retrive your API_KEY from config.json
def retrieve_apikey():
    with open('api_key.txt') as f:
         return f.readlines()[0]
        
# Write a file
def log(ext = 'json', **kwargs):
    for name, data in kwargs.items():
        with open(f'json/{name}.{ext}', 'w') as outfile:
            json.dump(data, outfile, sort_keys = True, indent = 4)

## Imagery functions

In [ ]:
# Save image
def save_image(image, name):
    cv2.imwrite(f'images/{name}', image)

def display_saved_image(name, pixel=750):
    display(HTML(f'<img style="float:left; max-width:{pixel}px; max-height:{pixel}px" src="images/{name}">'))
    
# Save and Display an image in notebook
def save_and_display_image(image, name, pixel=750):
    save_image(image, name)
    display_saved_image(name, pixel)

# Load image from response buffer
def load_image(r):
    np_array = np.frombuffer(r.content, np.uint8)
    return cv2.imdecode(np_array, cv2.IMREAD_COLOR)

# Make .gif : https://github.com/Zulko/moviepy/issues/744
def build_gif(filenames, name):
    clips = []
    fps = 12
    seconds = 3.5
    padding = .5
    padding_start = 0
    for filename in filenames:
        sequence = [filename for i in range(int(seconds*fps))]
        clip = ImageSequenceClip(sequence, fps=fps)
        if (padding_start > 0):
            clip = clip.set_start(padding_start).crossfadein(padding)
        clips.append(clip)
        padding_start += clip.duration - padding
    final = CompositeVideoClip(clips)
    final.write_gif(f'images/{name}.gif', fps=fps)
    clear_output()

## Functions to integrate API data

In [ ]:
# Extract xml response (for wmts)
def extract_xml(r):
    # Convert response to XML
    doc = r.content.decode('utf-8')
    return untangle.parse(doc)

# Extract template url from wmts xml
def extract_template(xml):
    return xml.Capabilities.Contents.Layer.ResourceURL['template']

# Format wmts url template with some parameters
def format_template(template, style, projection, zoom, col, row):
    formatted_template = re.sub('{[A-z]*}', '{}', template)
    return formatted_template.format(style, projection, zoom, col, row)

# Build a large tiled image
@Halo(text='Loading')
def build_large_tile(token, template, zoom, origin, n_tiles):
    # Fill the tile matrix
    tile_matrix = []
    for i in range(n_tiles):
        tile_matrix.append([])
        row = origin.y + i
        for j in range(n_tiles):
            col = origin.x + j
            # Request one tile
            formatted_template = format_template(template, 'rgb', 'EPSG3857', zoom, col, row)
            r = requests.get(formatted_template,
                headers = {'Authorization':f'Bearer {token}'})
            tile = load_image(r)
            tile_matrix[i].append(tile)
    # Build the image with the tile matrix (concat all tiles)
    return cv2.vconcat([cv2.hconcat(h) for h in tile_matrix])

# Request images and build gif with date
@Halo(text='Building gif...')
def request_images_and_build_gif(token, features, zoom, origin, n_tile, name):
    filenames = []
    last_acquisitionDate = None
    i = -1
    for f in features:
        if (f['properties']['acquisitionDate'][:10] != last_acquisitionDate):
            try:
                # Template
                r = requests.get(f['_links']['wmts']['href'],
                         headers={'Authorization':f'Bearer {token}'})
                t = extract_template(extract_xml(r))
                # Large tile
                im = build_large_tile(token, t, zoom, origin, n_tile)
                # Write date
                last_acquisitionDate = f['properties']['acquisitionDate'][:10]
                cv2.rectangle(im,(5, im.shape[0] - 5),(130, im.shape[0] - 35),(255,255,255),-1)
                cv2.rectangle(im,(5, im.shape[0] - 5),(130, im.shape[0] - 35),(0,0,0),1)
                cv2.putText(im, last_acquisitionDate, (15, im.shape[0] - 15), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0))
                # Save
                i += 1
                save_image(im, f'forGif/{i}.jpg')
                filenames.append(f'images/forGif/{i}.jpg')
                print(f'.{i}\t{last_acquisitionDate}')
            except:
                pass
    build_gif(filenames, name)

## Pretty print functions

In [ ]:
# Result from search request
def print_search(r_search):
    # Extract the total number of available images from response
    nb = int(r_search['totalResults'])
    # Print some details
    print(f'There are {nb} images available over this location.')
    print('index\t- date\t\t\tconstellation\tcloud\tformat')
    for i in range(nb):
        p = r_search['features'][i]['properties']
        print(f".{i}\t- {p['acquisitionDate'][:19].replace('T', ' ')}\t{p['constellation']}\t\t{p['cloudCover']}\t{p['format']}")

# Result from wmts request
def print_wmts(xml):
    Contents = xml.Capabilities.Contents
    # Style
    print(f'Styles')
    for Style in Contents.Layer.Style :
        identifier = Style.ows_Identifier.cdata
        default = '(default)' if Style['isDefault'] else ''
        print(f'- {identifier} {default}')
    # Projections
    print('\nProjection')
    for TileMatrixSet in Contents.TileMatrixSet:
        title = TileMatrixSet.ows_Title.cdata
        identifier = TileMatrixSet.ows_Identifier.cdata
        zoom_min = TileMatrixSet.TileMatrix[0].ows_Identifier.cdata
        zoom_max = TileMatrixSet.TileMatrix[-1].ows_Identifier.cdata
        while len(title) < 40: title += ' ' 
        print(f'- {title} : id {identifier}, zoom {zoom_min} ~ {zoom_max}')
    # Template
    template = Contents.Layer.ResourceURL['template']
    print(f'\nTemplate URL\n{template}')

# One Atlas Data API
- Access **token**
- Search for images with **opensearch**
- Access tiles with **WMTS** protocol
- Access full raw informations with **getBuffer**

# Access Token
1. Get your **api_key** at https://data.api.oneatlas.airbus.com/api-keys
2. Request an **access_token**

In [ ]:
api_key = retrieve_apikey()

r = requests.post('https://authenticate.foundation.api.oneatlas.airbus.com/auth/realms/IDP/protocol/openid-connect/token',
                  headers={'Content-Type':'application/x-www-form-urlencoded'},
                  data={'apikey':api_key, 'grant_type':'api_key', 'client_id':'IDP'})

token = r.json()['access_token']

print('Success: Access granted !')

# Search
1. Use **opensearch** with custom parameters
2. Display image **quicklook**

## Opensearch
Filter & Sort images
- Latitude & Longitude
- Satellite : Pléiades or SPOT
- Acquisition date
- ...

In [ ]:
params = {
    'lat': 39.812939, 
    'lon': -121.402845,
    'format':'image/jp2',
    'constellation':'PHR',
    'acquisitionDate':'[2016,2019[',
    'sortBy':'acquisitionDate'
}

r = requests.get('https://search.oneatlas.geoapi-airbusds.com/api/v1/opensearch',
                 headers={'Content-Type':'application/json', 'Authorization':f'Bearer {token}'},
                 params=params)

r_search = r.json()

print_search(r_search)

## Quicklook
1. Extract image from previous list
2. Request the quicklook

In [ ]:
feature = r_search['features'][-1]

r = requests.get(feature['_links']['quicklook']['href'],
                 headers={'Authorization':f'Bearer {token}'})

quicklook = load_image(r)

save_and_display_image(quicklook, 'quicklook.jpg')

# Tiles
- Extract **WMTS** parameters
- Get **tiles**
- Example : create a time comparison (.gif)

## WMTS

Standard protocol Web Map Tile Service

This request returns a **xml**

Extracting the template **url** from this xml

In [ ]:
r = requests.get(feature['_links']['wmts']['href'],
                 headers={'Authorization':f'Bearer {token}'})

xml = extract_xml(r)

template = extract_template(xml)

print_wmts(xml)

## One tile
WMTS parameters
- style
- projection (tile matrix set) : here we choose **WebMercator** projection EPSG:3857
- zoom (tile matrix)
- column & row

Get the tile column & row with our converter https://tceife2-idp-prod-oneatlasdata.appspot.com/guides/oneatlas-data/g-service-view/#get-tiles-for-a-web-mercator-tile-matrix-set

Python users can access this with the **mercantile** library 

In [ ]:
# Basic parameters
style = 'rgb'
projection = 'EPSG3857' # WebMercator projection EPSG:3857
zoom = 14 # around 10m resolution

# Get col & row match lon & lat & zoom according to the WebMercator projection
origin = mercantile.tile(params['lon'], params['lat'], zoom)
col = origin.x
row = origin.y

# Format the wmts template
formatted_template = format_template(template, style, projection, zoom, col, row)

# Request the tile using it
r = requests.get(formatted_template,
                 headers={'Authorization':f'Bearer {token}'})

tile = load_image(r)
save_and_display_image(tile, 'tile.jpg')

## Large tiled image

Request n*n tiles and build a large tiled image

Tile size = 256 pixels

In [ ]:
n_tiles = 2 # large image = n*n tiles

large_tile = build_large_tile(token, template, zoom, origin, n_tiles)

save_and_display_image(large_tile, 'large_tile.jpg')

## In time

Timelapse of our AOI

In [ ]:
features = r_search['features']

request_images_and_build_gif(token, features, zoom, origin, n_tiles, 'in_time')

display_saved_image('in_time.gif')